# Importing the Libraries needed

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

# Building the AI

In [2]:
class neural_network(nn.Module):
  def __init__(self, state_size, action_size, seed=47):
    super(neural_network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)   #created a variable fc1 to denote the first fully connected layer between input layer (state_size: i.e 8 as per our experiment) and hidden layer (64 nodes)
    self.fc2 = nn.Linear(64, 64)           #created a variable fc2 to denote the second fully connected layer between hidden layer (64 nodes) and a second hidden layer (64 nodes)
    self.fc3 = nn.Linear(64, action_size)  #created a variable fc3 to denote the final fully connected layer between the 2nd hidden layer (64 nodes) and the output layer (action_size i.e 4 as per our experiment)

  def forward(self, state):  # function to implement the forward propogation of out neural_network
    x = self.fc1(state)
    x = F.relu(x)         #data forward propogation from input layer to 1st hidden layer with relu activation function
    x = self.fc2(x)
    x = F.relu(x)         #data forward propogation from 1st hidden layer to 2nd hidden layer with relu activation function
    return self.fc3(x)    #data forward propogation from 2nd hidden layer to output layer with relu activation function and return as the output of the forward propogation

# Training the AI

### Setting up the environment

In [3]:
import gymnasium as gym                                      # import the gymnasium library
env = gym.make("LunarLander-v3")                             # set up the Lunar Lander environment
state_shape = env.observation_space.shape                    # define the shape of the input i.e the observation space (rows x cols)
state_size = env.observation_space.shape[0]                  # gather the size of the input i.e the number of rows in our input
number_actions = env.action_space.n                          # number of actions i.e the total actions the agent can take (the output size)

print('State shape {}, State size {}, Number of actions {}'.format(state_shape, state_size, number_actions))

State shape (8,), State size 8, Number of actions 4


### Initializing the hyperparameters

In [4]:
learning_rate = 5e-4
minibatch_size = 100
gamma = 0.99  # Is the discount factor: if the value is closer to 0, the agent will look at the shrot term reward and might not train for the entire case: If it is closer to 1, then agent trains with the long term factor in mind
replay_buffer_size = int(1e5)   # the number of experiments of scenarios the agent needs to look at to train the model
interpolation_param = 1e-3    #Parameter used to make sure the agent is training in a continous state

### Implementing Experience replay

In [5]:
class ReplayMemory(object):
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # runs on gpu if torch detects a gpu else runs on cpu; Gpu trains faster
    self.capacity = capacity                    # variable created to access the memory buffer and here the max size of memory buffer is taken into consideration
    self.memory = []                            # An empty list used to store the experiences to update the agent and its action in the environment

  def push(self, event):                # method to create experiences and store them
    self.memory.append(event)           # appending the events that take place into our memory list
    if len(self.memory) > self.capacity:    # if the memory exceeds our available max memory then we delete the oldest event stored
      del self.memory[0]

  def sample(self, batch_size):        # method to pick out random samples of our experiences
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    ''' np.vstack --> to vertically stack the elements : e[0] --> the state values in the experience : torch.from_numpy() --> convert the values into pytorch tensors : .float() --> convert the datatype to float
    .to(self.device) --> To move the stack of states into the GPU or CPU to train the tensors '''
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)       # e[1] --> the action values in experience and the datatype is long int
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN Class

In [6]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")        # runs on gpu if torch detects a gpu else runs on cpu; Gpu trains faster
    self.state_size = state_size
    self.action_size = action_size                                                      # declare the state_size and the action_size
    self.local_qnetwork = neural_network(state_size, action_size).to(self.device)       # creating a local q learning network using the neural_network class we created earlier and sending it to the device
    self.target_qnetwork = neural_network(state_size, action_size).to(self.device)      # creating a target q learning network using the neural_network class we created earlier and sending it to the device
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)   # creating the Adam optimizer for our q learning network which takes the parameters of the local q network
    self.memory = ReplayMemory(replay_buffer_size)                                      # initializing the memory using the ReplayMemory class created
    self.time_step = 0                                                                  # creating a variable to store the time step to keep track of the training

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.time_step = (self.time_step + 1) % 4
    if self.time_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, gamma)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, gamma):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (gamma * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.softupdate(self.local_qnetwork, self.target_qnetwork, interpolation_param)

  def softupdate(self, local_model, target_model, interpolation_param):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_param * local_param.data + (1.0 - interpolation_param) * target_param.data)

### Initializing the DQN agent

In [7]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [8]:
number_episodes = 2000
max_num_timesteps_per_episode = 1000
epsilon_start_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_start_value
scores_on_100_episodes = deque(maxlen = 100)

In [9]:
for episode in range(1, number_episodes+1):
  state, _ = env.reset()
  score = 0
  for t in range(max_num_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\t Average Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 ==0:
    print('\rEpisode {}\t Average Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 250.0:
    print('\nEnvironment solved in {:d} episode!\t Average Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	 Average Score: -172.58
Episode 200	 Average Score: -137.83
Episode 300	 Average Score: -79.088
Episode 400	 Average Score: -20.30
Episode 500	 Average Score: 33.341
Episode 600	 Average Score: 138.54
Episode 700	 Average Score: 165.93
Episode 800	 Average Score: 186.59
Episode 900	 Average Score: 178.85
Episode 1000	 Average Score: 218.80
Episode 1100	 Average Score: 237.72
Episode 1200	 Average Score: 213.55
Episode 1300	 Average Score: 239.52
Episode 1400	 Average Score: 241.17
Episode 1500	 Average Score: 233.39
Episode 1600	 Average Score: 231.60
Episode 1644	 Average Score: 251.33
Environment solved in 1644 episode!	 Average Score: 251.33


# Visualizing the results

In [10]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
  env = gym.make(env_name, render_mode = 'rgb_array')
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action.item())
  env.close()
  imageio.mimsave('lunar_landing.mp4', frames, fps=60)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
  mp4list = glob.glob('*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    display(HTML(data='''<video alt="test" autoplay loop controls style="height: 400px;"><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))
  else:
    print("could not find video")

show_video()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
